In [90]:
import pandas as pd

# 读取数据

## 训练集&测试集处理

In [91]:
data_org_dir = 'data/data_tencent/'
data_prep_dir = 'data/preprocess/'

In [92]:
# 特征数据集
df_user_feature = pd.read_csv(data_org_dir + 'userFeature.csv')
df_ad_feature = pd.read_csv(data_org_dir + 'adFeature.csv')
# 训练集
df_train = pd.read_csv(data_org_dir + 'train.csv')
# 测试集
df_test1 = pd.read_csv(data_org_dir + 'test1.csv')
df_test2 = pd.read_csv(data_org_dir + 'test2.csv')

In [93]:
df_user_feature.columns

Index(['LBS', 'age', 'appIdAction', 'appIdInstall', 'carrier',
       'consumptionAbility', 'ct', 'education', 'gender', 'house', 'interest1',
       'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
       'marriageStatus', 'os', 'topic1', 'topic2', 'topic3', 'uid'],
      dtype='object')

In [94]:
df_user_feature.head()

,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,ct,education,gender,house,...,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3,uid
0,950.0,4,NaN,NaN,1,2,3 1,7,2,NaN,...,52 100 72 131 116 11 71 12 8 113 28 73 6 132 9...,664359 276966 734911 103617 562294,11395 79112 115065 77033 36176,NaN,11,2,9826 105 8525 5488 7281,9708 5553 6745 7477 7150,NaN,26325489
1,803.0,2,NaN,NaN,1,1,3 1,2,1,NaN,...,NaN,338851 361151 542834 496283 229952,80263 39618 53539 180 38163,NaN,5 13,1,4391 9140 5669 1348 4388,9401 7724 1380 8890 7153,NaN,1184123
2,927.0,1,NaN,NaN,3,1,3 1,5,1,NaN,...,77 72 80 116 101 13 1 109 8 50 6 42 76 9 46 36...,746140 695808 126355 771775 411511,105115 71378 41409 74061 44005,NaN,13 10,1,1502 5488 9826 2187 8088,5005 9154 2756 5612 4209,NaN,76072711
3,486.0,4,NaN,NaN,3,1,1 3,7,1,NaN,...,100 80 92 37 116 13 47 4 71 8 50 28 98 115 6 4...,283399 402245 734509 654027 32061,74010 32918 67882 116802 20957,NaN,11,1 2,1619 7342 3064 9213 8525,810 2438 5659 1844 9262,NaN,63071413
4,112.0,5,NaN,NaN,3,2,3 1 4 2,6,1,NaN,...,131 116 13 8 6 132 42 9 59 18 58 64 129 103,562294 157603 589706 657719 495672,62764 97803 89066 55545 74061,NaN,11,1,477 9826 5808 644 2747,5483 2199 5424 1511 7751,NaN,81294159


In [95]:
df_ad_feature.head()

,aid,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,177,8203,76104,1500666,59,282,0,6
1,2050,19441,178687,245165,53,1,0,6
2,1716,5552,158101,1080850,35,27,113,9
3,336,370,4833,119845,22,67,113,9
4,671,45705,352827,660519,42,67,0,4


In [96]:
print('训练集长度: ', len(df_train))
print(df_train.columns)

训练集长度:  8798814
Index(['aid', 'uid', 'label'], dtype='object')


In [97]:
# 正负样本比例
df_train.label.value_counts()

0    8376853
1     421961
Name: label, dtype: int64

In [98]:
"""
实验时候无需用到测试集
"""
# 插入test字段，0代表训练集，1代表测试集1，2代表测试集2
# df_test1['test'] = 1
# df_test2['test'] = 2

# 合并两组测试集合
# df_test = pd.concat([df_test1, df_test2], axis=0, ignore_index=True)
# df_test['label'] = -1

# df_test.test.value_counts()

# 合并测试集和训练集
# df_train_test = pd.concat([df_train, df_test], axis=0, ignore_index=True)
# 训练集的test列为0
# df_train_test['test'].fillna(0, inplace=True)

# df_train_test.label.value_counts()

'\n实验时候无需用到测试集\n'

## 关联特征数据

In [99]:
# 关联用户特征
df_data = pd.merge(df_train, df_user_feature, on=['uid'], how='left')
# 关联广告特征
df_data = pd.merge(df_data, df_ad_feature, on=['aid'], how='left')

In [100]:
df_data.isna().sum()

aid                         0
uid                         0
label                       0
LBS                        64
age                         0
appIdAction           8665951
appIdInstall          8620544
carrier                     0
consumptionAbility          0
ct                          0
education                   0
gender                      0
house                 7207472
interest1              798391
interest2             2962021
interest3             8566189
interest4             8665859
interest5             2210891
kw1                    882574
kw2                    299051
kw3                   8396313
marriageStatus              0
os                          0
topic1                 750086
topic2                 316545
topic3                8395203
advertiserId                0
campaignId                  0
creativeId                  0
creativeSize                0
adCategoryId                0
productId                   0
productType                 0
dtype: int

In [101]:
df_data.fillna(-1, inplace=True)

df_data['label'] = df_data['label'].astype(float)


## 构建样本数据
由于本次比赛没有给出任何时间特征，所以构造转化率特征时很容易产生数据穿越，从而过拟合。为了解决数据穿越，可以采用分块求转化率，或者贝叶斯平滑等方法。值得注意的是转化率利用预处理所得的分块标签独立出一个分块验证集不加入统计，其余分块做dropout交叉统计，测试集则用全部训练集数据进行统计，很好的解决的数据穿越问题。

In [102]:
aid_lst = list(df_data['aid'].unique())
len(aid_lst)

173

In [52]:
# params initialazation
n_parts = 5  # 数据块的数量
data_idx = []
for i in range(n_parts):
    data_idx.append([])

for aid in aid_lst:  # 保证在每一块数据里都有全部的广告id
    # 截取该广告id下的全部数据
    df_data_aid = df_data[df_data['aid'] == aid]
    for k in [0, 1]:  # 对正负样本loop
        idx_lst = list(df_data_aid[df_data_aid['label'] == k].sample(frac=1, random_state=10).index)  # sample是为了能够shuffle一下
        cut = [0]
        for i in range(n_parts):
            # 将该部分数据平均分成5块，每一块数据的end idx
            end_idx = int((i + 1) * len(idx_lst) / n_parts) + 1
            cut.append(end_idx)
        for j in range(n_parts):
            # 将每一块数据的row idx拼接到list里
            idx2append = idx_lst[cut[j]: cut[j + 1]]
            data_idx[j].extend(idx2append)

In [55]:
# 构建数据集分块标记
n_parts_col = pd.Series()
for n in range(n_parts):
    nth_part = pd.Series(n + 1, index=data_idx[n])
    n_parts_col = n_parts_col.append(nth_part)
print(len(n_parts_col) == len(df_data))

df_data.insert(0, 'n_parts', list(pd.Series(df_data.index).map(n_parts_col).values))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


5215782    1
7794089    1
4243595    1
8322191    1
2006005    1
          ..
5636562    5
178748     5
8768063    5
2007290    5
4715286    5
Length: 8798814, dtype: int64


In [60]:
df_data.n_parts.value_counts()

1    1759974
3    1759770
4    1759756
2    1759756
5    1759558
Name: n_parts, dtype: int64

In [66]:
# 训练集和验证集切割+储存
df_valid = df_data[df_data['n_parts'] == 1]
df_train = df_data[df_data['n_parts'] != 1]

df_train.to_csv(data_prep_dir + 'train_data.csv', index=False)
df_valid.to_csv(data_prep_dir + 'validation_data.csv', index=False)

print('=================')
print('训练数据集长度为{}'.format(len(df_train)))
# print('正负样本数量为: ', df_train.label.value_counts())
print('包含item数量为: ', int(df_train.aid.nunique()))

print('=================')
print('验证数据集长度为{}'.format(len(df_valid)))
# print('正负样本数量为: ', df_valid.label.value_counts())
print('包含item数量为: ', int(df_valid.aid.nunique()))


# 储存各数据块
length = 0
for i in range(n_parts):
    df_data_sub = df_data[df_data['n_parts'] == i + 1]
    print('=================')
    print('数据块{}长度为{}'.format(i + 1, len(df_data_sub)))
#     print('正负样本数量为: ', df_data_sub.label.value_counts())
    print('包含item数量为: ', int(df_data_sub.aid.nunique()))
    df_data_sub.to_csv(data_prep_dir + 'train_chunk_{}.csv'.format(i + 1), index=False)

df_data.to_csv(data_prep_dir + 'train_feat_merge.csv', index=False)

print('All datasets are saved!!')

训练数据集长度为7038840
包含item数量为:  173
验证数据集长度为1759974
包含item数量为:  173
数据块1长度为1759974
包含item数量为:  173
数据块2长度为1759756
包含item数量为:  173
数据块3长度为1759770
包含item数量为:  173
数据块4长度为1759756
包含item数量为:  173
数据块5长度为1759558
包含item数量为:  173
All data are saved!!


## 构建小规模实验数据

In [113]:
df_mini = df_data[df_data['n_parts'] == 5]
df_mini.drop('n_parts', axis=1, inplace=True)
len(df_mini)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


527717

In [114]:


# params initialazation
n_parts = 3  # 数据块的数量
data_idx = []
for i in range(n_parts):
    data_idx.append([])

for aid in aid_lst:  # 保证在每一块数据里都有全部的广告id
    # 截取该广告id下的全部数据
    df_data_aid = df_mini[df_mini['aid'] == aid]
    for k in [0, 1]:  # 对正负样本loop
        idx_lst = list(df_data_aid[df_data_aid['label'] == k].sample(frac=1, random_state=10).index)  # sample是为了能够shuffle一下
        cut = [0]
        for i in range(n_parts):
            # 将该部分数据平均分成5块，每一块数据的end idx
            end_idx = int((i + 1) * len(idx_lst) / n_parts) + 1
            cut.append(end_idx)
        for j in range(n_parts):
            # 将每一块数据的row idx拼接到list里
            idx2append = idx_lst[cut[j]: cut[j + 1]]
            data_idx[j].extend(idx2append)
            
# 构建数据集分块标记
n_parts_col = pd.Series()
for n in range(n_parts):
    nth_part = pd.Series(n + 1, index=data_idx[n])
    n_parts_col = n_parts_col.append(nth_part)
print(len(n_parts_col) == len(df_mini))

df_mini.insert(0, 'n_parts', list(pd.Series(df_mini.index).map(n_parts_col).values))

# 训练集和验证集切割+储存
df_valid = df_mini[df_mini['n_parts'] == 1]
df_train = df_mini[df_mini['n_parts'] != 1]
print('train_data_mini: ', len(df_train))
print('validation_data_mini: ', len(df_valid))
df_train.to_csv(data_prep_dir + 'train_data_mini.csv', index=False)
df_valid.to_csv(data_prep_dir + 'validation_data_mini.csv', index=False)
df_mini.to_csv(data_prep_dir + 'train_feat_merge_mini.csv', index=False)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


True
train_data_mini:  351578
validation_data_mini:  176139


In [115]:
df_mini.to_csv(data_prep_dir + 'train_feat_merge_mini.csv', index=False)

# 数据探索

In [67]:
df_data.head()

,n_parts,aid,uid,label,LBS,age,appIdAction,appIdInstall,carrier,consumptionAbility,...,topic1,topic2,topic3,advertiserId,campaignId,creativeId,creativeSize,adCategoryId,productId,productType
0,1,699,78508957,0.0,576.0,1,-1,-1,0,1,...,4908 9745 7070 4203 3292,-1,-1,1082,295940,731679,59,13,0,6
1,1,1991,3637295,0.0,353.0,3,-1,-1,1,0,...,8445 6762 7608 523 7688,4518 862 3643 702 5247,-1,702,42104,1441131,53,10,4669,11
2,2,1119,19229018,0.0,94.0,4,-1,-1,0,0,...,941 4674 9735 8953 5352,-1,-1,3993,63752,798752,59,10,19256,11
3,1,2013,79277120,0.0,48.0,2,-1,-1,1,1,...,4538 6790 8850 3422 1940,2979 1877 358 666 4166,-1,6937,186348,1427984,35,89,3791,9
4,1,692,41528441,0.0,333.0,1,-1,-1,1,1,...,4535 5590 47 5570 2078,856 4587 1877 392 9103,-1,6946,296367,455396,59,24,3794,11


## 正负样本比例

In [68]:
df_data.label.value_counts()
# pos 421961 / neg 8376853

0.0    8376853
1.0     421961
Name: label, dtype: int64

## 用户特征insight

In [69]:
df_data.columns

Index(['n_parts', 'aid', 'uid', 'label', 'LBS', 'age', 'appIdAction',
       'appIdInstall', 'carrier', 'consumptionAbility', 'ct', 'education',
       'gender', 'house', 'interest1', 'interest2', 'interest3', 'interest4',
       'interest5', 'kw1', 'kw2', 'kw3', 'marriageStatus', 'os', 'topic1',
       'topic2', 'topic3', 'advertiserId', 'campaignId', 'creativeId',
       'creativeSize', 'adCategoryId', 'productId', 'productType'],
      dtype='object')


### uid 用户id
- 含义：用户id


### label 样本标记
- 含义：正负样本标记 train_pos 1 / train_neg 0 / test -1


### test 是否测试集合
- 含义：测试数据集合标记，1表示测试集1，2表示测试集2

### LBS 用户位置
- 含义：Location Based Services，相当于一个用户位置的分类


In [19]:
df_data.LBS.nunique()  # Location Based Services

864

### age 年龄
- 离散特征，无缺失值

In [127]:
df_data.age.unique()

array([1, 3, 4, 2, 5, 0])

### appIdAction
- 含义：
- 序列特征，string形式，各元素用空格分隔，缺失值用-1表示

In [128]:
df_data.appIdAction.unique()

array([-1, '2183 4807 2911', '3269 562 4041 4973 1088 5266', ...,
       '1559 5606 27 1266 2283 5531 725 1968 2964 4106 2892 2755',
       '3287 4058 562 2168',
       '517 3292 2482 1584 4572 3269 872 4324 4114 3457 5983 2192'],
      dtype=object)

### appIdInstall 安装的APP
- 序列特征，string形式，各元素用空格分隔，缺失值用-1表示

In [73]:
df_data.appIdInstall.unique()[:3]

array([-1,
       '60806 8899 7764 50585 19763 45857 50704 12226 45134 47751 11938 19039 13158 47826 59290',
       '10876 45028 6071 29193 55984 26035 37826 42554 58506 23206 64678 40044 25207 4980 12004 61664 45505 1123 3237 57061 47366 62385 24522 56616 34691 22759 63809 30699 60283 52013 21970 54768 9174 47169 49587 50237 18814 46808 37518 59311 42747 14237 46809 28827 59290 19020 57849 13132 14380 39261 19039 22164 11245 144 40038 58031 64692 42892 41664 35015 57514 12035 16267'],
      dtype=object)

### carrier运营商
- 离散特征，无缺失情况

In [133]:
df_data.carrier.unique()

array([0, 1, 3, 2])

### consumptionAbility 消费能力等级
- 离散特征，无缺失值

In [134]:
df_data.consumptionAbility.unique()

array([1, 0, 2])

### ct
- 含义：
- 序列特征，无缺失值，各元素用空格分隔，缺失值用-1表示

In [135]:
df_data.ct.unique()

array(['0', '3', '3 1', '1 3 4', '4 1 3', '3 1 2', '1 3', '3 4 1 2',
       '3 2 4 1', '3 1 4', '3 4 1', '1 4 3', '1', '1 3 2 4', '3 1 4 2',
       '1 3 2', '3 2 4', '2 3 1', '4 3 1', '3 4', '3 2 1 4', '2 1 3',
       '3 2', '3 4 2', '3 2 1', '1 2 3', '1 3 4 2', '1 4 3 2', '1 2 3 4',
       '4', '2 3 1 4', '3 1 2 4', '1 2', '2 3', '1 4', '1 4 2 3', '4 3',
       '1 2 4 3', '2 4 1 3', '2 1 3 4', '4 1 3 2', '4 3 2', '4 3 1 2',
       '4 1', '4 2 3 1', '2', '4 1 2 3', '2 4 3 1', '2 4 3', '2 3 4',
       '3 4 2 1', '2 4', '2 3 4 1', '2 1', '4 2 1 3', '2 1 4 3', '4 2 3',
       '1 4 2', '1 2 4', '4 2 1', '4 3 2 1', '4 1 2', '2 4 1', '4 2',
       '2 1 4'], dtype=object)

### education 受教育等级
- 离散特征，无缺失值，

In [136]:
df_data.education.unique()

array([1, 7, 2, 6, 5, 0, 3, 4])

### gender 性别

In [72]:
df_data.gender.unique()

array([1, 2, 0])

### house 有没有房产
- 离散特征，有缺失值，用-1填充

In [74]:
df_data.house.unique()

array([-1.,  1.])

### interest1 to interest5 用户兴趣
- 序列特征，空格分割，有缺失值，用-1填充

In [75]:
df_data.interest1.unique()[:2]

array(['70 100 48 49 23 36 11', '93 109 75 29 59 49 6 5 73 46 22 36 11 3'],
      dtype=object)

### kw1 to kw3 关键词（搜索？）
- 序列特征，空格分割，有缺失值，用-1填充

In [76]:
df_data.kw1.unique()[:2]

array(['124532 762727 771871 8561 610048',
       '531499 542834 303322 503318 68211'], dtype=object)

### marriageStatus 婚姻状况（？？？）
- 序列特征，不定长，无缺失值

In [77]:
df_data.marriageStatus.unique()

array(['10', '13', '13 10', '11', '13 9', '0', '12 13', '15', '13 15',
       '6 13', '2 13', '5 13', '6 13 9', '14', '12 13 9', '12 13 10',
       '2 13 9', '5 13 9', '3', '8', '15 10', '5 13 10', '6 13 10',
       '13 15 10', '13 10 9', '2 13 10'], dtype=object)

### os 手机系统
- 序列特征，不定长，无缺失值

In [79]:
df_data.os.unique()

array(['0', '1', '2', '1 2', '2 1'], dtype=object)

### topic1 to topic3
- 序列特征，定长，缺失值-1填充

In [81]:
df_data.topic1.unique()[:3]

array(['4908 9745 7070 4203 3292', '8445 6762 7608 523 7688',
       '941 4674 9735 8953 5352'], dtype=object)

## 广告特征insight

### aid 广告id
含义：广告id

In [71]:
df_data.aid.nunique()

173

### advertiserId 广告商id
- 无缺失

In [82]:
df_data.advertiserId.nunique()

79

### campaignId 活动id

In [85]:
df_data.campaignId.nunique()

138

### creativeId

In [84]:
df_data.creativeId.nunique()

173

### creativeSize

In [86]:
df_data.creativeSize.nunique()

15

### adCategoryId 广告分类id

In [87]:
df_data.adCategoryId.nunique()

40

### productId 广告产品id

In [88]:
df_data.productId.nunique()

33

### productType 广告产品分类

In [89]:
df_data.productType.nunique()

4